In [4]:
# TBD 1 : logger 추가
# TBD 2: flask github 참고, method, class, 파일의 맨 윗단 마다 pydoc 형식으로 달기
# TBD 3: 축약어를 자제할것 (특히 변수)

# -------------------------
#   To-do
# -------------------------
# 1. add logger
# 2. make image drawer overlay mask on image
# 3. make iterable
# 4. make verbose turn on and off
# 5. write pydoc

# tensorflow Module
import tensorflow as tf
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import ReduceLROnPlateau

# python basic Module
import os
from datetime import datetime

import numpy as np
import segmentation_models as sm

from gan_module.data_loader.medical_segmentation_data_loader import DataLoader
from gan_module.model.build_model import build_generator_non_unet as build_generator
from gan_module.util import custom_loss
from gan_module.util.custom_loss import WeightedRegionLoss, TverskyLoss, dice_score, dice_loss
from gan_module.config import CONFIG


custom_loss.AXIS = [1, 2]
USE_GPU = True
# set GPU memory growth allocation
if USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)


class UnetSegmentation:
    def __init__(
        self,
        generator_power=32,
        depth=None,
        generator_learning_rate=1e-4,
        on_memory=True,
        code_test=False
    ):

        img_shape = CONFIG["img_shape"]
        input_channels = CONFIG["input_channels"]
        output_channels = CONFIG["output_channels"]

        input_img_shape = (*img_shape, input_channels)

        # Configure data loader
        self.dataset_name = "glomerulus_0.65_512_remove_peel_split_man"
        self.data_loader = DataLoader(
            dataset_name=self.dataset_name,
            on_memory=on_memory,
            code_test=code_test,
            config_dict=CONFIG
        )

        self.loaded_data_index = {
            "train": np.arange(self.data_loader.data_length["train"]),
            "valid": np.arange(self.data_loader.data_length["valid"])
        }

        # Number of filters in the first layer of G and D
        self.generator_power = generator_power
        self.generator_learning_rate = generator_learning_rate
        generator_optimizer = Nadam(self.generator_learning_rate)

        self.generator = sm.Unet(backbone_name=BACKBONE, input_shape=(
            None, None, 3), classes=1, activation='sigmoid')
        # loss = sm.losses.bce_dice_loss
        self.generator.compile(
            loss=TverskyLoss(per_image=True),
            optimizer=generator_optimizer,
            metrics=[dice_score],
        )

    def train(self, epochs, batch_size=10, start_epoch=0):

        start_time = datetime.now()

        reduce_lr = LearningRateScheduler(self.learning_rate_scheduler)
        save_c = ModelCheckpoint(
            "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
            monitor='val_loss',
            verbose=0,
            save_best_only=False,
            save_weights_only=True,
            mode='min')

        reduceLROnPlat = ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.1,
            patience=10,
            verbose=1,
            mode="auto",
            min_delta=0.0001,
            cooldown=5,
            min_lr=1e-7)
        csv_logger = CSVLogger('./U_net/log.csv', append=False, separator=',')
        x_train = preprocess_input(self.data_loader.loaded_data_object["train"]["input"])
        y_train = self.data_loader.loaded_data_object["train"]["output"]
        x_val = preprocess_input(self.data_loader.loaded_data_object["valid"]["input"])
        y_val = self.data_loader.loaded_data_object["valid"]["output"]
        self.generator.fit(
            x = x_train,
            y = y_train,
            validation_data=(x_val, y_val),
            batch_size=batch_size, epochs=epochs,
            callbacks=[reduceLROnPlat, save_c, csv_logger],
            initial_epoch=start_epoch
        )

        elapsed_time = datetime.now() - start_time
        print(f"elapsed_time : {elapsed_time}")

    def learning_rate_scheduler(self, epoch,
                                schedule_list=None, exponent=0.2,
                                warm_up=True, warm_up_epoch=10):
        step = 0
        if warm_up and epoch < warm_up_epoch:
            new_learning_rate = self.generator_learning_rate * \
                ((epoch + 1) / warm_up_epoch)
        else:
            if schedule_list is None:
                schedule_list = [30, 100, 175, 250, 325]
            for step, target_epoch in enumerate(schedule_list):
                if target_epoch > epoch:
                    break
                else:
                    continue
            new_learning_rate = self.generator_learning_rate * \
                (exponent**(step))

        return new_learning_rate


In [5]:
generator_lr = 1e-4
dataset_name = "glomerulus_0.65_512_remove_peel_split_man"
batch_size = 4
g_lr = generator_lr * batch_size
gan = UnetSegmentation(generator_power=8, depth=None, generator_learning_rate=g_lr, on_memory=True, code_test=True)

In [6]:
#gan.find_error = True
#gan.find_error_epoch = 5
gan.train(epochs=325, batch_size=batch_size, start_epoch=0)

Epoch 1/325


KeyboardInterrupt: 